# CryptBot
> Another C++ bot

- toc: true 
- badges: true
- categories: [cryptbot,botnet,yara,config]


## Overview

This is another C++ bot! According to [Malpedia](https://malpedia.caad.fkie.fraunhofer.de/details/win.cryptbot)...

> A typical infostealer, capable of obtaining credentials for browsers, crypto currency wallets, browser cookies, credit cards, and creates screenshots of the infected system. All stolen data is bundled into a zip-file that is uploaded to the c2.

## Samples
Samples available on [UnpacMe](https://www.unpac.me/results/008a502c-9308-4cea-b2b1-1bdce7f01b7b#/)
- Packed `7ccda59528c0151bc9f11b7f25f8291d99bcf541488c009ef14e2a104e6f0c5d`
- Unpacked `cfbecf45c083efffff6d3000972a66cddb2f26d5c1845a697351b132e65049e0`

## References 

- [40,000 CryptBot Downloads per Day: Bitbucket Abused as Malware Slinger](https://www.gdatasoftware.com/blog/2020/02/35802-bitbucket-abused-as-malware-slinger)
- [CryptBot Infostealer: Malware Analysis](https://any.run/cybersecurity-blog/cryptbot-infostealer-malware-analysis/)



## Analysis

This is a some in-memory string that was captured in a [joe sandbox run](https://www.joesandbox.com/analysis/805090/0/html)

```
 Content-Length: Content-Type: multipart/form-data;boundary=httphttpstrue<>S-1-5-18[<apis.google.com>]/[<443>][<www.google.com>][<"facebook">][<www.facebook.com>][<TEMP>][<APPDATA>][<LOCALAPPDATA>][<USERPROFILE>];[<ExternalDownload>][<Anti>][<true>][<UserAgent>][<UAC>]runas[<NTFS>][<Prefix>][<UID: >][<UserName: >][<ComputerName: >][<Info>][<OS: >][<DateTime: >][<UserAgent: >][<Keyboard Languages: >][<Display Resolution: >][<CPU: >][<RAM: >][<GPU: >][<isGodMod: yes>][<isGodMod: no>][<isAdmin: yes>][<isAdmin: no>][<Installed software:>][<Disk:>][<Process:>][<Screenshot>][<InfoFile>][<ScreenFile>][<PasswordFile>][<ChromeDBFolder>][<ChromeExt>][<WalletFolder>][<_Chrome_profile>][<EdgeDB>][<EdgeDBFolder>][<EdgeExt>][<_Edge_profile>][<Desktop>][<DesktopFolder>][<.txt>][<Wallet>]_test.err://[<80>][<OK>][< /c >][<cmd>][<open>][<MessageAfterEnd>][<System Error>][<The application was unable to start correctly (0xc000007b). Click OK to close the application.>][<DeleteAfterEnd>][< /c timeout  -t  5  && del ">]
```


### Yara Rule
Some of these strings exist in the binary unencrypted so we can use them for a yara rule

```c
rule cryptbot {
    
    strings:
        $s1 = "UID:"
        $s2 = "UserName:"
        $s3 = "ComputerName:"
        $s4 = "DateTime:"
        $s5 = "UserAgent:"
        $s6 = "Keyboard Languages:"
        $s7 = "Display Resolution:"
        $s8 = "CPU:"
        $s9 = "RAM:"
        $s10 = "GPU:"
        $s11 = "isGodMod: yes"
        $s12 = "isGodMod: no"
        $s13 = "isAdmin: yes"
        $s14 = "isAdmin: no"
        $s15 = "Installed software:"
    condition:
        all of them
}
```

### Config 

Finding a ref to the config in the code.
```
80 3D ?? ?? ?? ?? 09                    cmp     byte ptr ptr_config, 9 ; "PSJigdSdi8"
B9 00 01 00 00                          mov     ecx, 100h
```

```
B9 00 01 00 00                          mov     ecx, 100h
80 3D ?? ?? ?? ?? 09                    cmp     byte ptr a7m1fqxrljy, 9 ; "7m1fqXrLJy"
```





In [35]:
import re
import pefile
import struct

file_data = open('/tmp/cryptbot.bin', 'rb').read()
pe = pefile.PE(data = file_data)
image_base = pe.OPTIONAL_HEADER.ImageBase

text_data = None

for s in pe.sections:
    if b'.text' == s.Name[:5]:
        text_data = s.get_data()
        break
        
assert text_data is not None


eggs = [
        rb'\x80\x3D(....)\x09\xB9\x00\x01\x00\x00',
        rb'\xB9\x00\x01\x00\x00\x80\x3D(....)\x09'
       ]

candidate_offsets = []

for egg in eggs:
    for m in re.finditer(egg, text_data, re.DOTALL):
        try:
            candidate_va = struct.unpack('<I', m.group(1))[0]
            candidate_offset = pe.get_offset_from_rva(candidate_va - image_base)
            candidate_offsets.append(candidate_offset)
        except:
            print(f"failed for group {m.group(1)}")
            pass

        


In [36]:
def xor_decrypt(data, key):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i % len(key)])
    return bytes(out)


def get_config(data, offset):
    key = data[offset:].split(b'\x00')[0]
    assert 5  < len(key) < 20
    config_data_enc = data[offset + len(key) + 1:] 
    return xor_decrypt(config_data_enc, key)


config_data = None

for candidate_offset in candidate_offsets:
    try:
        tmp_config = get_config(file_data, candidate_offset)
        if tmp_config[:4] == b'http':
            config_data = tmp_config
            break
    except:
        pass

assert config_data is not None


# config parse
config_array = []
for a in config_data.split(b'\x00'):
    if not a.isascii():
        break
    config_array.append(a)

c2 = config_array[0].decode('utf-8')
settings = []

for config_entries in config_array[1:]:
    for entry in config_entries.split(b'<>\r\n'):
        if len(entry) == 0:
            continue
        settings.append({'key':entry.split(b'<>_<>')[0].decode('utf-8'), 'value':entry.split(b'<>_<>')[1].decode('utf-8')})

print(c2)
print(settings)



http://erniku42.top/gate.php;
[{'key': 'CookiesEdge', 'value': 'false'}, {'key': 'HistoryEdge', 'value': 'false'}, {'key': 'HistoryFirefox', 'value': 'false'}, {'key': 'EdgeDB', 'value': 'true'}, {'key': 'Edge', 'value': 'false'}, {'key': 'Files', 'value': 'false'}, {'key': 'Opera', 'value': 'false'}, {'key': 'CookiesOpera', 'value': 'false'}, {'key': 'HistoryOpera', 'value': 'false'}, {'key': 'Screenshot', 'value': 'true'}, {'key': 'Chrome', 'value': 'false'}, {'key': 'Info', 'value': 'true'}, {'key': 'HistoryChrome', 'value': 'false'}, {'key': 'ChromeDB', 'value': 'true'}, {'key': 'Wallet', 'value': 'true'}, {'key': 'ChromeExt', 'value': 'true'}, {'key': 'Firefox', 'value': 'false'}, {'key': 'CookiesChrome', 'value': 'false'}, {'key': 'FirefoxDB', 'value': 'true'}, {'key': 'CookiesFirefox', 'value': 'false'}, {'key': 'Desktop', 'value': 'true'}, {'key': 'EdgeExt', 'value': 'true'}, {'key': 'CookiesFile', 'value': '_AllCookies.txt'}, {'key': 'HistoryFile', 'value': '_AllHistory.txt'},

In [39]:

def get_config_from_file(file_path):
    file_data = open(file_path, 'rb').read()
    pe = pefile.PE(data = file_data)
    image_base = pe.OPTIONAL_HEADER.ImageBase

    text_data = None

    for s in pe.sections:
        if b'.text' == s.Name[:5]:
            text_data = s.get_data()
            break

    assert text_data is not None


    eggs = [
            rb'\x80\x3D(....)\x09\xB9\x00\x01\x00\x00',
            rb'\xB9\x00\x01\x00\x00\x80\x3D(....)\x09'
           ]

    candidate_offsets = []

    for egg in eggs:
        for m in re.finditer(egg, text_data, re.DOTALL):
            try:
                candidate_va = struct.unpack('<I', m.group(1))[0]
                candidate_offset = pe.get_offset_from_rva(candidate_va - image_base)
                candidate_offsets.append(candidate_offset)
            except:
                print(f"failed for group {m.group(1)}")
                pass

    assert len(candidate_offsets) != 0

    config_data = None

    for candidate_offset in candidate_offsets:
        try:
            tmp_config = get_config(file_data, candidate_offset)
            if tmp_config[:4] == b'http':
                config_data = tmp_config
                break
        except:
            pass
    
    assert config_data is not None


    # config parse
    config_array = []
    for a in config_data.split(b'\x00'):
        if not a.isascii():
            break
        config_array.append(a)

    c2 = config_array[0].decode('utf-8')
    settings = []

    for config_entries in config_array[1:]:
        for entry in config_entries.split(b'<>\r\n'):
            if len(entry) == 0:
                continue
            settings.append({'key':entry.split(b'<>_<>')[0].decode('utf-8'), 'value':entry.split(b'<>_<>')[1].decode('utf-8')})

    assert len(settings) != 0 

    final_config = {'c2':c2, 'settings':settings}
    return final_config

get_config_from_file('/tmp/cryptbot.bin')

{'c2': 'http://erniku42.top/gate.php;',
 'settings': [{'key': 'CookiesEdge', 'value': 'false'},
  {'key': 'HistoryEdge', 'value': 'false'},
  {'key': 'HistoryFirefox', 'value': 'false'},
  {'key': 'EdgeDB', 'value': 'true'},
  {'key': 'Edge', 'value': 'false'},
  {'key': 'Files', 'value': 'false'},
  {'key': 'Opera', 'value': 'false'},
  {'key': 'CookiesOpera', 'value': 'false'},
  {'key': 'HistoryOpera', 'value': 'false'},
  {'key': 'Screenshot', 'value': 'true'},
  {'key': 'Chrome', 'value': 'false'},
  {'key': 'Info', 'value': 'true'},
  {'key': 'HistoryChrome', 'value': 'false'},
  {'key': 'ChromeDB', 'value': 'true'},
  {'key': 'Wallet', 'value': 'true'},
  {'key': 'ChromeExt', 'value': 'true'},
  {'key': 'Firefox', 'value': 'false'},
  {'key': 'CookiesChrome', 'value': 'false'},
  {'key': 'FirefoxDB', 'value': 'true'},
  {'key': 'CookiesFirefox', 'value': 'false'},
  {'key': 'Desktop', 'value': 'true'},
  {'key': 'EdgeExt', 'value': 'true'},
  {'key': 'CookiesFile', 'value': '_Al

In [42]:
from pathlib import Path

for file in Path('/tmp/samples/').glob('*'):
    print(file)
    try:
        config = get_config_from_file(file)
        print(config.get('c2'), 'None')
    except Exception as e:
        print("ERROR")
        print(e)


/tmp/samples/909ce699e4a2680687b65ca3d4ff8cd24a410c05ceda581741e17aa429b12983
http://ernjxs12.top/gate.php; None
/tmp/samples/c20a40b230e26207a392fc196ecf818cd41c400aff19cd124bcb6a831ea5c1b9
ERROR

/tmp/samples/f8675c4a0fa94fb4d0bf070bc171ddfd732143bf1fba1847c537ef43ea00f6a6
http://ewzpak62.top/gate.php; None
/tmp/samples/de9216d23a71a911d38028743d3d7af93da2e29774601cd96a4bf8ead842058f
http://lahwsg62.top/gate.php; None
/tmp/samples/4aee291fc52bf99bf92331c7c6eeedc87d5c2e7a00291af522a6e2249794cf76
http://lahrom42.top/gate.php; None
/tmp/samples/bcbbba94fb3901541a97a0cf5c95209cf4520eb9d731691ce28e7995459b8eac
http://xjuxjt32.top/gate.php; None
/tmp/samples/c6338e4242cf07ce63beb69365c79b6101cbde5e6101a04d4398b2433ac20786
http://olsodv72.top/gate.php; None
/tmp/samples/004b0ef1531d232cd379ade9a64548618ea5652c9b3c6adf8fd9614a6b64d752
ERROR

/tmp/samples/addc404a3575d31882a52539d77c379c2efd6bd7d3c9adde1cf0480b128cd8ad
ERROR

/tmp/samples/c7348d5113889d4527e6f2deab6620ad36c6fa693aff1f604d1233